In [ ]:

import pandas as pd

#loading dataset
df = pd.read_csv("ppi.csv")

#making sure positions are sorted
df = df.sort_values(["uniprot_id", "aa_ProtPosition"])

#grouping and reconstructing sequences
protein_sequences = (df.groupby("uniprot_id")["sequence"].apply(lambda x: "".join(x)).reset_index())


print(protein_sequences.head())


   uniprot_id                                           sequence
0      A0A010  MLAAEAANRDHVTRCVAQTGGSPDLVAHTAALRLYLRVPHFLTEWT...
1  A0A024RAV5  MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...
2  A0A0D5YE19  MQHPTSTDIQRVREFLLDLQARICAGLEQQEKAGGGTAEFIIDDWE...
3  A0A0F4FI39  MRNWLVALASLLLLAGCEKPAEQVHLSGPTMGTTYNIKYIQQPGIA...
4  A0A0H2VDD2  MTPWLLFGAGGKGVGARTLELALAEQRPVVAVIRHADAATKLAQQG...


In [ ]:
#making fasta file
with open("proteins.fasta", "w") as f:
    for _, row in protein_sequences.iterrows():
        f.write(f">{row['uniprot_id']}\n")
        f.write(f"{row['sequence']}\n")

In [ ]:
#verifying sequences, that there is no empty one
protein_sequences["sequence"].apply(len).describe()

count    228.000000
mean     285.745614
std      155.935083
min       38.000000
25%      159.250000
50%      255.500000
75%      388.250000
max      695.000000
Name: sequence, dtype: float64

In [ ]:

blast = pd.read_csv("blast_results.tsv", sep="\t", header=None)

blast.columns = [
    "query", "subject", "identity", "alignment_length",
    "mismatches", "gap_opens", "q_start", "q_end",
    "s_start", "s_end", "evalue", "bitscore"
]

#keeping the hit with highest bit score
best_hits = blast.sort_values("bitscore", ascending=False)\
                 .groupby("query")\
                 .first()\
                 .reset_index()

print(best_hits)

          query                subject  identity  alignment_length  \
0    A0A024RAV5   sp|P79800|RASK_MELGA    98.936               188   
1    A0A0D5YE19   sp|B7VMW3|HEM6_VIBA3    63.961               308   
2    A0A0F4FI39   sp|A5F5Y3|APBE_VIBC3   100.000               334   
3    A0A0H2VDD2   sp|Q04304|YMY0_YEAST    28.402               169   
4    A0A0H3JPM4   sp|Q2FWV6|SCIN_STAA8    31.304               115   
..          ...                    ...       ...               ...   
204      Q9Y6K8   sp|Q9Y6K8|KAD5_HUMAN   100.000               562   
205      Q9Z2X2  sp|Q9Z2X2|PSD10_MOUSE   100.000               231   
206      S7UWA0   sp|P48510|DSK2_YEAST    31.311               412   
207      U3J0P1  sp|P00705|LYSC1_ANAPL    98.639               147   
208      W6EQP0   sp|O68252|PCEA_SULMU    99.800               501   

     mismatches  gap_opens  q_start  q_end  s_start  s_end         evalue  \
0             2          0        1    188        1    188  1.320000e-138   
1    

In [ ]:
# checking if there is 25% identity between proteins in the dataset
import pandas as pd

blast = pd.read_csv("self_blast.tsv", sep="\t", header=None)

blast.columns = [
    "query", "subject", "identity", "alignment_length",
    "mismatches", "gap_opens", "q_start", "q_end",
    "s_start", "s_end", "evalue", "bitscore"
]



In [ ]:
#removing proteins that match protein to itself => 100% similarity
blast = blast[blast["query"] != blast["subject"]]


In [ ]:
#computing convergance additionaly to identity
#loading original lengths
df = pd.read_csv("ppi.csv")
lengths = df.groupby("uniprot_id").size().to_dict()

blast["query_length"] = blast["query"].map(lengths)
blast["coverage"] = blast["alignment_length"] / blast["query_length"]


In [ ]:
#filtering above 25% identity and 0.8 homology
homologs = blast[(blast["identity"] >= 25) & (blast["coverage"] >= 0.8)]

print(homologs)



       query subject  identity  alignment_length  mismatches  gap_opens  \
126   B2TEV0  P13081    27.820               133          81          6   
143   B4RLV9  Q0I165    25.688               218         137          7   
298   L7N6F8  P17901    28.000                75          41          3   
615   P13081  B2TEV0    27.820               133          81          6   
768   P40029  P0A3Q9    39.103               156          85          3   
797   P41784  P46075    25.000                80          51          3   
944   Q0I165  B4RLV9    25.688               218         137          7   
1020  Q1MVD4  P11349    31.429                70          28          3   
1024  Q1MVD4  W6EQP0    26.027                73          36          3   
1025  Q1MVD4  W6EQP0    25.301                83          52          2   
1335  Q889N2  P0A734    26.866                67          45          3   

      q_start  q_end  s_start  s_end        evalue  bitscore  query_length  \
126         1    129 

In [ ]:
#filtering with the e value alos 
filtered = blast[(blast["identity"] >= 25) &  (blast["coverage"] >= 0.8) & (blast["evalue"] <= 1e-5)]

print(filtered)



      query subject  identity  alignment_length  mismatches  gap_opens  \
143  B4RLV9  Q0I165    25.688               218         137          7   
768  P40029  P0A3Q9    39.103               156          85          3   
944  Q0I165  B4RLV9    25.688               218         137          7   

     q_start  q_end  s_start  s_end        evalue  bitscore  query_length  \
143        5    207       14    221  8.600000e-17      70.1           227   
768       19    174        4    149  9.190000e-36     120.0           184   
944       14    221        5    207  8.940000e-17      70.1           236   

     coverage  
143  0.960352  
768  0.847826  
944  0.923729  


Pairs of proteins that are homologous Q0I165 - B4RLV9  and  P40029 - P0A3Q9m, the rest 224 proteins is unique. Data set has minimal redundency 